In [6]:
import torch
from torch.autograd import Variable

import torch.nn as nn
import numpy as np

class DDNN(nn.Module) : 
    def __init__(self, layers):
        super(DDNN, self).__init__()

        input_dim = 9
        output_dim = 3

        h1, h2, h3 = layers[1], layers[2], layers[3]
        h4, h5  = layers[4], layers[5]

        # Layer Strucutre
        self.l1 = nn.Linear(input_dim, h1)
        self.l2 = nn.Linear(h1, h2)
        self.l3 = nn.Linear(h2, h3)
        self.l4 = nn.Linear(h3, h4)
        self.l5 = nn.Linear(h4, h5)
        self.l6 = nn.Linear(h5, output_dim)
        # self.BatchNorm1 = nn.BatchNorm1d(h1)
        # self.BatchNorm2 = nn.BatchNorm1d(h2)


        # Define Activation Function 
        # Use Softplus for smoother result (compared with ReLU)
        self.act1 = nn.Softplus()
        self.act2 = nn.Softplus()
        self.act3 = nn.Softplus()
        self.act4 = nn.Softplus()
        self.act5 = nn.Softplus()

        # Use ReLU for MATLAB Compatibility (Manually changes to Softplus in MATLAB is needed)
        # self.act1 = nn.ReLU()
        # self.act2 = nn.ReLU()
        # self.act3 = nn.ReLU()
        # self.act4 = nn.ReLU()
        # self.act5 = nn.ReLU()

        

    def forward(self, x):
        
        x = self.l1(x)
        x = self.act1(x)
        x = self.l2(x)
        x = self.act2(x)
        x = self.l3(x)
        x = self.act3(x)
        x = self.l4(x)
        x = self.act4(x)
        x = self.l5(x)
        x = self.act5(x)
        x = self.l6(x)

        return x


config = {
    'n_epochs': 50,                    # maximum number of epochs
    'batch_size': 16,                   # mini-batch size for dataloader
    'optimizer': 'Adam',                # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                   # hyper-parameters for the optimizer (depends on which optimizer you are using)
        'lr': 0.0005433177089293607                     # learning rate of Adam
    },
    'layers': np.array([9, 189, 119, 9, 53, 85, 3], dtype='int64'),    # layers of NN architecture
    'early_stop': 200,                  # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pt'      # your model will be saved here
}

model = DDNN(config['layers'])
file_config = {'save_path' : 'models/model_PGNN.pt'}
ckpt = torch.load(file_config['save_path'], map_location='cpu')  # Load your best model
model.load_state_dict(ckpt)
dummy_input = Variable(torch.randn(1,1,9))
print(ckpt)

torch.onnx.export(model, 
    dummy_input, 
    "PGNN_noisefree_softplus_matlab.onnx",
    export_params=True,  # store the trained parameter weights inside the model file 
    opset_version=9)

OrderedDict([('l1.weight', tensor([[-0.0049,  0.2394,  1.5218,  ..., -0.3944,  0.9606, -0.0289],
        [-0.0050,  1.6082, -1.5638,  ...,  0.4160, -0.2517,  0.2136],
        [-0.0050,  0.3273,  0.9517,  ...,  1.1212, -2.7600, -1.3541],
        ...,
        [-0.0050, -0.7095, -2.9865,  ..., -1.0293, -0.7689, -1.4483],
        [-0.0036, -1.7491,  1.1301,  ..., -1.3502, -2.6372, -1.3950],
        [-0.0086,  0.5905, -1.2763,  ..., -2.8561, -0.1365, -0.4780]])), ('l1.bias', tensor([-3.7508e-01,  3.9478e-01, -6.8742e-01, -6.4702e-01,  2.3803e-01,
         2.4849e-01,  4.1825e-01,  4.8524e-01,  2.5979e-03, -8.0682e-01,
        -1.0906e+00, -3.3737e-01, -9.4143e-02,  2.9798e-01,  3.0296e-01,
        -2.7705e-01, -3.4931e-01,  3.2368e-01,  3.8321e-01,  2.7644e-01,
        -5.1015e-01, -7.0496e-01,  4.1499e-01, -4.0107e-01, -4.4250e-01,
         8.2763e-02, -2.1640e-01, -1.8291e-01, -3.7059e-01,  2.5726e-01,
        -6.5593e-01,  2.3909e-01,  4.9817e-01, -2.2989e-01, -3.0652e-01,
        -8.447